# Practice VI
***
* Gonzalez Chacon Monica
* Lopez Salazar Esmeralda Leticia
* Rodriguez Nuñez Diego Eduardo

In [2]:
!python -m spacy download es_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 31.4 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [1]:
import pandas as pd
import spacy
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import f1_score , ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re
from itertools import combinations
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay, f1_score

In [2]:
nlp = spacy.load('es_core_news_sm')

In [3]:
data = pd.read_excel('Rest_Mex_2022.xlsx')
data

,Title,Opinion,Polarity,Attraction
0,Pésimo lugar,"Piensen dos veces antes de ir a este hotel, te...",1,Hotel
1,No vayas a lugar de Eddie,Cuatro de nosotros fuimos recientemente a Eddi...,1,Restaurant
2,Mala relación calidad-precio,seguiré corta y simple: limpieza\n- bad. Tengo...,1,Hotel
3,Minusválido? ¡No te alojes aquí!,Al reservar un hotel con multipropiedad Mayan ...,1,Hotel
4,Es una porqueria no pierdan su tiempo,"No pierdan su tiempo ni dinero, venimos porque...",1,Hotel
...,...,...,...,...
30207,Verdadera joya arquitectónica,"Es una construcción majestuosa, creo que de la...",5,Attractive
30208,Romántico,Muy al estilo de Romeo y Julieta es este sitio...,5,Attractive
30209,Parece un castillo,Ideal para subir las escalinatas y divisar su ...,5,Attractive
30210,Imperdible,"Es imperdible, de ahí puedes ver muy bien la c...",5,Attractive


In [4]:
data['Features'] = data['Title'].astype(str) + ' ' + data['Opinion'].astype(str)
X = data['Features'].fillna('')
y = data['Polarity']
data

,Title,Opinion,Polarity,Attraction,Features
0,Pésimo lugar,"Piensen dos veces antes de ir a este hotel, te...",1,Hotel,Pésimo lugar Piensen dos veces antes de ir a e...
1,No vayas a lugar de Eddie,Cuatro de nosotros fuimos recientemente a Eddi...,1,Restaurant,No vayas a lugar de Eddie Cuatro de nosotros f...
2,Mala relación calidad-precio,seguiré corta y simple: limpieza\n- bad. Tengo...,1,Hotel,Mala relación calidad-precio seguiré corta y s...
3,Minusválido? ¡No te alojes aquí!,Al reservar un hotel con multipropiedad Mayan ...,1,Hotel,Minusválido? ¡No te alojes aquí! Al reservar u...
4,Es una porqueria no pierdan su tiempo,"No pierdan su tiempo ni dinero, venimos porque...",1,Hotel,Es una porqueria no pierdan su tiempo No pierd...
...,...,...,...,...,...
30207,Verdadera joya arquitectónica,"Es una construcción majestuosa, creo que de la...",5,Attractive,Verdadera joya arquitectónica Es una construcc...
30208,Romántico,Muy al estilo de Romeo y Julieta es este sitio...,5,Attractive,Romántico Muy al estilo de Romeo y Julieta es ...
30209,Parece un castillo,Ideal para subir las escalinatas y divisar su ...,5,Attractive,Parece un castillo Ideal para subir las escali...
30210,Imperdible,"Es imperdible, de ahí puedes ver muy bien la c...",5,Attractive,"Imperdible Es imperdible, de ahí puedes ver mu..."


In [5]:
#esto ignoralo xd
#y = y.str.replace(r'\s+', ' ', regex=True)

Division del dataset

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=0)

In [7]:
def NormalizarTexto(texto, aplicar):
    texto = texto.lower() if 'text_cleaning' in aplicar else texto

    doc = nlp(texto)
    tokens = []
    for token in doc:
        if not token.is_punct and not token.is_space:
            if 'stop_words' in aplicar and token.is_stop:
                continue
            tokens.append(token.lemma_ if 'lemmatization' in aplicar else token.text)
    return ' '.join(tokens) if 'tokenization' in aplicar else ' '.join([texto])

In [8]:
procesos = ['tokenization', 'stop_words', 'lemmatization', 'text_cleaning']
combos = [list(c) for i in range(1,len(procesos)+1) for c in combinations(procesos, i)]

resultados = {}

for combo in combos:
    print(f'Procesando combinacion: {combo}')
    X_train_norm = X_train.apply(lambda x: NormalizarTexto(x, aplicar=combo))
    X_test_norm = X_test.apply(lambda x: NormalizarTexto(x, aplicar=combo))
    resultados[str(combo)] = (X_train_norm[:5], X_test_norm[:5])

for combo, (tran_sample, test_sample) in resultados.items():
    print(f'Combinación: {combo}')
    print(f'Train: {tran_sample}')
    print(f'Test: {test_sample}')
    print("*"*50)

Procesando combinacion: ['tokenization']
Procesando combinacion: ['stop_words']
Procesando combinacion: ['lemmatization']
Procesando combinacion: ['text_cleaning']
Procesando combinacion: ['tokenization', 'stop_words']
Procesando combinacion: ['tokenization', 'lemmatization']
Procesando combinacion: ['tokenization', 'text_cleaning']
Procesando combinacion: ['stop_words', 'lemmatization']
Procesando combinacion: ['stop_words', 'text_cleaning']
Procesando combinacion: ['lemmatization', 'text_cleaning']
Procesando combinacion: ['tokenization', 'stop_words', 'lemmatization']
Procesando combinacion: ['tokenization', 'stop_words', 'text_cleaning']
Procesando combinacion: ['tokenization', 'lemmatization', 'text_cleaning']
Procesando combinacion: ['stop_words', 'lemmatization', 'text_cleaning']
Procesando combinacion: ['tokenization', 'stop_words', 'lemmatization', 'text_cleaning']
Combinación: ['tokenization']
Train: 18594    El mejor también en el lockdown También ahora ...
19690    Vacacion

In [9]:
representaciones = {}

for combo in combos:
    print(f'\nGenerando representación para combinación: {combo}')

    X_test_norm = X_test.apply(lambda x: NormalizarTexto(x, aplicar=combo))
    X_train_norm = X_train.apply(lambda x: NormalizarTexto(x, aplicar=combo))

    #Binarizada
    vectorizer_binary = CountVectorizer(binary=True)
    X_train_binary = vectorizer_binary.fit_transform(X_train_norm)
    X_test_binary = vectorizer_binary.transform(X_test_norm)
    representaciones[f'{combo} + Binarized'] = (X_train_binary, X_test_binary)
    print(f'Combinacion: {combo} + Binarized\nPreview:',X_train_binary[:5].toarray())

    #Frecuencia
    vectorizer_freq = CountVectorizer()
    X_train_freq = vectorizer_freq.fit_transform(X_train_norm)
    X_test_freq = vectorizer_freq.transform(X_test_norm)
    representaciones[f'{combo} + Frequency'] = (X_train_freq, X_test_freq)
    print(f'Combinacion: {combo} + Frequency\nPreview:',X_train_freq[:5].toarray())

    #TF-IDF
    vectorizer_tfidf = TfidfVectorizer()
    X_train_tfidf = vectorizer_tfidf.fit_transform(X_train_norm)
    X_test_tfidf = vectorizer_tfidf.transform(X_test_norm)
    representaciones[f'{combo} + TF-IDF'] = (X_train_tfidf, X_test_tfidf)
    print(f'Combinacion: {combo} + TF-IDF\nPreview:',X_train_tfidf[:5].toarray())

    #TF-IDF + SVD
    svd = TruncatedSVD(n_components=100, random_state=42)
    X_train_svd = svd.fit_transform(X_train_tfidf)
    X_test_svd = svd.transform(X_test_tfidf)
    representaciones[f'{combo} + TF-IDF + SVD'] = (X_train_svd, X_test_svd)
    print(f'Combinacion: {combo} + TF-IDF + SVD\nPreview:',X_train_svd[:3])


Generando representación para combinación: ['tokenization']
Combinacion: ['tokenization'] + Binarized
Preview: [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
Combinacion: ['tokenization'] + Frequency
Preview: [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
Combinacion: ['tokenization'] + TF-IDF
Preview: [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
Combinacion: ['tokenization'] + TF-IDF + SVD
Preview: [[ 2.41621234e-01  4.79882854e-02  1.03664829e-02  4.66655714e-02
   3.14382104e-02  6.57808836e-02 -4.58912796e-03  5.73569045e-02
  -5.23138241e-02 -1.19605446e-02  2.64313301e-02 -2.19615443e-02
  -4.27485833e-02  1.96757517e-02 -1.96125377e-02 -5.24901289e-02
  -3.53814128e-02  3.95225358e-03  6.06474060e-02  1.47009924e-02
   3.14056836e-03  3.15836327e-03 -1.35173881e-02 -4.18870576e-02
  -8.00366930e-03 -1.

Entrenar y evaluar clasificadores

In [10]:
def TrainandEvaluate(classifier,X_train,X_test,y_train,y_test):
    classifier.fit(X_train, y_train)
    y_pred = classifier.predict(X_test)
    f1 = f1_score(y_test, y_pred, average='macro')
    return f1, y_pred

In [11]:
classifiers ={
    'Logistic Regression': LogisticRegression(max_iter=1000),
    'Naive Bayes': MultinomialNB(alpha = 0.5),
    'Neural Network': MLPClassifier(hidden_layer_sizes=(200,100)),
}

In [ ]:
results = {}

for clf_name, clf in classifiers.items():
    results[clf_name] = {}
    for com_rep_name, (X_train_rep, X_test_rep) in representaciones.items():
        if clf_name == "Naive Bayes" and "TF-IDF" in com_rep_name:
            print(f"Skipping {clf_name} with {com_rep_name}")
            continue
        print(f'\nEvaluando {clf_name} con combincion:{com_rep_name}')
        try:
            f1, y_pred = TrainandEvaluate(clf,X_train_rep,X_test_rep,y_train,y_test)
            results[clf_name][com_rep_name] = f1
            print(f'F1 Score para {clf_name} con {com_rep_name}: {f1:.3f}')
            print(f'\nReporte de Clasificación para {clf_name} con {com_rep_name}:')
            print(classification_report(y_test, y_pred, zero_division=0))
        except Exception as e:
            print(f'Error al evaluar {clf_name} con {com_rep_name}: {e}')


Evaluando Logistic Regression con combincion:['tokenization'] + Binarized
F1 Score para Logistic Regression con ['tokenization'] + Binarized: 0.452

Reporte de Clasificación para Logistic Regression con ['tokenization'] + Binarized:
              precision    recall  f1-score   support

           1       0.49      0.32      0.38       104
           2       0.28      0.21      0.24       145
           3       0.41      0.35      0.38       422
           4       0.41      0.38      0.40      1163
           5       0.84      0.88      0.86      4209

    accuracy                           0.72      6043
   macro avg       0.49      0.43      0.45      6043
weighted avg       0.71      0.72      0.71      6043


Evaluando Logistic Regression con combincion:['tokenization'] + Frequency
F1 Score para Logistic Regression con ['tokenization'] + Frequency: 0.479

Reporte de Clasificación para Logistic Regression con ['tokenization'] + Frequency:
              precision    recall  f1-score

c:\Users\dern9\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


F1 Score para Logistic Regression con ['tokenization', 'lemmatization'] + Frequency: 0.492

Reporte de Clasificación para Logistic Regression con ['tokenization', 'lemmatization'] + Frequency:
              precision    recall  f1-score   support

           1       0.52      0.47      0.49       104
           2       0.33      0.25      0.28       145
           3       0.45      0.40      0.42       422
           4       0.44      0.37      0.40      1163
           5       0.84      0.89      0.86      4209

    accuracy                           0.73      6043
   macro avg       0.51      0.48      0.49      6043
weighted avg       0.72      0.73      0.72      6043


Evaluando Logistic Regression con combincion:['tokenization', 'lemmatization'] + TF-IDF
F1 Score para Logistic Regression con ['tokenization', 'lemmatization'] + TF-IDF: 0.446

Reporte de Clasificación para Logistic Regression con ['tokenization', 'lemmatization'] + TF-IDF:
              precision    recall  f1-scor

c:\Users\dern9\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


F1 Score para Logistic Regression con ['tokenization', 'lemmatization', 'text_cleaning'] + Frequency: 0.498

Reporte de Clasificación para Logistic Regression con ['tokenization', 'lemmatization', 'text_cleaning'] + Frequency:
              precision    recall  f1-score   support

           1       0.54      0.49      0.52       104
           2       0.36      0.28      0.31       145
           3       0.43      0.37      0.40       422
           4       0.43      0.37      0.40      1163
           5       0.84      0.89      0.86      4209

    accuracy                           0.73      6043
   macro avg       0.52      0.48      0.50      6043
weighted avg       0.71      0.73      0.72      6043


Evaluando Logistic Regression con combincion:['tokenization', 'lemmatization', 'text_cleaning'] + TF-IDF
F1 Score para Logistic Regression con ['tokenization', 'lemmatization', 'text_cleaning'] + TF-IDF: 0.444

Reporte de Clasificación para Logistic Regression con ['tokenization', '

...

F1 Score para Neural Network con ['tokenization'] + TF-IDF + SVD: 0.411

Reporte de Clasificación para Neural Network con ['tokenization'] + TF-IDF + SVD:
              precision    recall  f1-score   support

           1       0.31      0.28      0.29       104
           2       0.23      0.21      0.22       145
           3       0.34      0.33      0.34       422
           4       0.37      0.37      0.37      1163
           5       0.83      0.84      0.83      4209

    accuracy                           0.69      6043
   macro avg       0.42      0.41      0.41      6043
weighted avg       0.68      0.69      0.68      6043


Evaluando Neural Network con combincion:['stop_words'] + Binarized
F1 Score para Neural Network con ['stop_words'] + Binarized: 0.451

Reporte de Clasificación para Neural Network con ['stop_words'] + Binarized:
              precision    recall  f1-score   support

           1       0.49      0.34      0.40       104
           2       0.29      0.19 

c:\Users\dern9\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


F1 Score para Neural Network con ['stop_words'] + TF-IDF + SVD: 0.392

Reporte de Clasificación para Neural Network con ['stop_words'] + TF-IDF + SVD:
              precision    recall  f1-score   support

           1       0.29      0.30      0.29       104
           2       0.20      0.18      0.19       145
           3       0.32      0.27      0.29       422
           4       0.35      0.36      0.36      1163
           5       0.82      0.83      0.82      4209

    accuracy                           0.68      6043
   macro avg       0.40      0.39      0.39      6043
weighted avg       0.67      0.68      0.67      6043


Evaluando Neural Network con combincion:['lemmatization'] + Binarized
F1 Score para Neural Network con ['lemmatization'] + Binarized: 0.461

Reporte de Clasificación para Neural Network con ['lemmatization'] + Binarized:
              precision    recall  f1-score   support

           1       0.52      0.37      0.43       104
           2       0.28      

c:\Users\dern9\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


F1 Score para Neural Network con ['lemmatization'] + TF-IDF + SVD: 0.405

Reporte de Clasificación para Neural Network con ['lemmatization'] + TF-IDF + SVD:
              precision    recall  f1-score   support

           1       0.30      0.34      0.32       104
           2       0.23      0.22      0.23       145
           3       0.30      0.29      0.30       422
           4       0.37      0.34      0.35      1163
           5       0.82      0.84      0.83      4209

    accuracy                           0.68      6043
   macro avg       0.41      0.41      0.41      6043
weighted avg       0.68      0.68      0.68      6043


Evaluando Neural Network con combincion:['text_cleaning'] + Binarized
F1 Score para Neural Network con ['text_cleaning'] + Binarized: 0.456

Reporte de Clasificación para Neural Network con ['text_cleaning'] + Binarized:
              precision    recall  f1-score   support

           1       0.53      0.38      0.44       104
           2       0.29

Los 3 mejores resultados por clasificador

In [ ]:
for clf_name, clf_scores in results.items():
    print(f'\nClasificador: {clf_name}')
    top_combos = sorted(clf_scores.items(), key=lambda x: x[1], reverse=True)[:3]

    for rank, (combo_name, f1) in enumerate(top_combos, start=1):
        print(f'{rank+1}. Combinación: {combo_name}, F1 Score: {f1:.3f}')

        X_train_rep, X_test_rep = representaciones[combo_name]
        clf = classifiers[clf_name]
        clf.fit(X_train_rep, y_train)
        y_pred = clf.predict(X_test_rep)

In [ ]:
data = []
for clf_name, clf_scores in results.items():
    top_combos = sorted(clf_scores.items(), key=lambda x: x[1], reverse=True)[:3]
    for combo_name, f1 in top_combos:
        data.append({'Classifier': clf_name, 'Representation': combo_name, 'F1 Score': f1})

df = pd.DataFrame(data)
#df_multi = df.set_index(['Classifier', 'Representation', 'F1 Score'])
#df_multi
df

,Classifier,Representation,F1 Score
0,Logistic Regression,"['tokenization', 'lemmatization', 'text_cleani...",0.627513
1,Logistic Regression,"['tokenization', 'lemmatization', 'text_cleani...",0.610700
2,Logistic Regression,"['tokenization', 'lemmatization'] + Binarized",0.609505
3,Naive Bayes,"['tokenization', 'stop_words', 'lemmatization'...",0.620603
4,Naive Bayes,"['tokenization', 'stop_words', 'lemmatization'...",0.614150
5,Naive Bayes,"['tokenization', 'stop_words'] + Binarized",0.605219
6,Decision Tree,"['stop_words', 'lemmatization'] + TF-IDF + SVD",0.510492
7,Decision Tree,['lemmatization'] + TF-IDF + SVD,0.504589
8,Decision Tree,['stop_words'] + TF-IDF + SVD,0.493866
9,SVM,"['tokenization', 'stop_words'] + TF-IDF + SVD",0.534118
